# importando librerias

In [1]:
import pandas as pd
import numpy as np
import json
import ast
from pandas import json_normalize
from textblob import TextBlob

# Australian User Reviews

In [2]:
# Leer el contenido del archivo
user_reviews = []
with open('../PI MLOps - STEAM/australian_user_reviews.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        user_reviews.append(ast.literal_eval(line))

# Crear el DataFrame anidado
df_reviews_anidado = pd.DataFrame(user_reviews)

In [3]:
df_reviews_anidado.head(2)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."


quito la columna que no voy a usar

In [4]:
# quito la columna de user_url
df_reviews_anidado=df_reviews_anidado.drop(['user_url'],axis=1)

In [5]:
df_reviews_anidado.head(2)


,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."


uso de explode

In [6]:
#usamos explode para desanidar:
df_reviews = df_reviews_anidado.explode('reviews', ignore_index=True)

In [7]:

df_reviews.head()

,user_id,reviews
0,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,76561197970982479,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,76561197970982479,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."


jsonNormalize para crear un nuevo df y concat pára unirlos

In [8]:
# Usa json_normalize para expandir las columnas del objeto JSON en el DataFrame
df_reviews2 = json_normalize(df_reviews['reviews'])

# Combina el DataFrame original con el DataFrame resultante de json_normalize
df_reviews = pd.concat([df_reviews[['user_id']], df_reviews2], axis=1)

In [9]:
#quito las columnas que no voy a usar
df_reviews=df_reviews.drop(['funny','last_edited','helpful'],axis=1)

In [10]:
df_reviews.head(2)


,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.


guardad el nuevo csv

In [11]:
#guardo el df final
df_reviews.to_csv('../fastapi/reviews_sin_SA.csv', index=False) 

In [12]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59333 non-null  object
 1   posted     59305 non-null  object
 2   item_id    59305 non-null  object
 3   recommend  59305 non-null  object
 4   review     59305 non-null  object
dtypes: object(5)
memory usage: 2.3+ MB


In [13]:
df_reviews=pd.read_csv('../fastapi/reviews_sin_SA.csv')

eliminar filas sin reviews

In [14]:
df_reviews.dropna(subset=['posted', 'item_id', 'recommend', 'review'],how='all', inplace=True)


analisis de sentimientos

In [15]:
def analisis_sentimiento(texto):
    if isinstance(texto, str):  # Verifica si es una cadena de texto
        blob = TextBlob(texto)
        polaridad = blob.sentiment.polarity

        # Mapeo de polaridad a la escala 0, 1, 2
        if polaridad > 0.1:
            return 2  # Positivo
        elif polaridad < -0.1:
            return 0  # Malo
        else:
            return 1  # Neutral
    else:
        return 1  # Valor 1 si la reseña no es una cadena válida

In [16]:
# Aplicar la función a la columna 'review' y crear la columna 'sentiment_analysis'
df_reviews['sentiment_analysis'] = df_reviews['review'].apply(analisis_sentimiento)

In [17]:
df_reviews=df_reviews.drop(['review'],axis=1)

extraemos los años de la columna posted

In [18]:
df_reviews['year'] = df_reviews['posted'].str.extract(r'(\d{4})')

In [19]:
df_reviews=df_reviews.drop(['posted'],axis=1)

relleno los NaN de 'year' con un numero que no voy a usar y los paso a datatime

In [20]:
df_reviews['year']=df_reviews['year'].fillna('1900')
df_reviews['year'] = pd.to_datetime(df_reviews['year'], format='%Y', errors='coerce').dt.year


In [23]:
df_reviews.head(4)

,user_id,item_id,recommend,sentiment_analysis,year
0,76561197970982479,1250.0,True,2,2011
1,76561197970982479,22200.0,True,2,2011
2,76561197970982479,43110.0,True,1,2011
3,js41637,251610.0,True,2,2014


In [24]:
#guardo el df final
df_reviews.to_csv('../fastapi/reviewsF.csv', index=False) 